In [3]:
# retina
%config InlineBackend.figure_format = 'retina'

In [5]:
import os
from pathlib import Path
import pickle
import numpy as np
from matplotlib import pyplot as plt
from astropy.io import fits
from astropy.table import Table
import eazy

from matplotlib import rcParams
plt.rcParams["font.size"] = 20
plt.rcParams["axes.labelsize"] = 25

WD = Path('/data1/hbahk/spherex-photoz/spherex-challenge/challenge')
os.chdir(WD)

from utils import make_eazy_filters_spherex

In [33]:
# let's see the data structure
hdu = fits.open(WD/'data/L3_COSMOS2020_Richard_2023DEC4_SPHEREx_fiducial102_more.fits')
hdu.info()

Filename: /data1/hbahk/spherex-photoz/spherex-challenge/challenge/data/L3_COSMOS2020_Richard_2023DEC4_SPHEREx_fiducial102_more.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     26   166041R x 8C   [K, D, D, PD(102), PD(102), D, D, D]   


In [36]:
cat = Table(hdu[1].data)
print(cat.colnames)
print(len(cat))

['source_id', 'x', 'y', 'fiducial_flux', 'fiducial_error', 'HSC_i_MAG', 'z_true', 'FLAG_ML']
166041


In [38]:
cat_reduced = cat['source_id', 'x', 'y', 'HSC_i_MAG', 'z_true', 'FLAG_ML']
cat_reduced.write(WD/'data/fiducial102_reduced.fits', overwrite=True)

In [46]:
cat_synphot = Table.read(WD/'data/L3_COSMOS2020_Richard_2023DEC4_SPHEREx_fiducial102_for_photoz.txt', format='ascii.no_header')

In [47]:
cat_synphot[:2]

col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,col41,col42,col43,col44,col45,col46,col47,col48,col49,col50,col51,col52,col53,col54,col55,col56,col57,col58,col59,col60,col61,col62,col63,col64,col65,col66,col67,col68,col69,col70,col71,col72,col73,col74,col75,col76,col77,col78,col79,col80,col81,col82,col83,col84,col85,col86,col87,col88,col89,col90,col91,col92,col93,col94,col95,col96,col97,col98,col99,col100,col101,col102,col103,col104,col105,col106,col107,col108,col109,col110,col111,col112,col113,col114,col115,col116,col117,col118,col119,col120,col121,col122,col123,col124,col125,col126,col127,col128,col129,col130,col131,col132,col133,col134,col135,col136,col137,col138,col139,col140,col141,col142,col143,col144,col145,col146,col147,col148,col149,col150,col151,col152,col153,col154,col155,col156,col157,col158,col159,col160,col161,col162,col163,col164,col165,col166,col167,col168,col169,col170,col171,col172,col173,col174,col175,col176,col177,col178,col179,col180,col181,col182,col183,col184,col185,col186,col187,col188,col189,col190,col191,col192,col193,col194,col195,col196,col197,col198,col199,col200,col201,col202,col203,col204,col205
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
180642,-12.93,10.04,4.18,10.46,12.45,10.53,-10.32,11.83,0.6,10.85,-5.49,9.34,2.98,10.72,13.86,9.59,-10.02,10.29,-6.82,10.42,27.23,9.13,-8.28,8.45,-3.51,6.44,2.49,8.21,10.66,8.29,-4.09,9.0,13.52,9.55,34.25,12.5,4.91,5.8,-6.65,6.88,0.72,6.89,4.82,7.01,-0.9,6.57,1.6,5.24,7.11,5.93,1.35,4.65,9.69,5.44,-0.38,6.23,-12.41,5.99,6.43,5.44,-4.11,6.47,-11.08,6.42,2.28,6.19,16.48,11.14,-4.16,6.91,-2.19,6.41,-3.11,10.02,-1.6,7.81,16.47,12.64,8.29,9.36,4.45,9.03,10.84,8.37,-2.91,8.04,7.52,7.34,-4.62,6.25,4.36,7.47,-1.3,7.47,-4.62,7.28,-1.75,7.18,3.17,7.23,1.13,9.37,-6.13,7.6,7.06,8.26,11.44,8.33,13.36,8.02,9.41,8.88,16.02,7.43,0.56,7.71,18.61,8.31,20.02,8.82,10.55,7.34,1.74,6.61,11.86,6.92,0.36,5.69,13.75,5.37,3.66,6.28,9.13,6.88,11.03,7.18,32.55,18.27,0.05,8.07,11.93,14.37,31.63,14.33,-17.03,14.37,13.86,14.41,-23.31,12.61,-2.0,14.43,5.6,11.1,9.79,13.73,1.69,15.94,2.03,16.12,-4.25,15.21,7.88,17.6,-16.08,17.71,6.25,17.38,-47.59,25.32,47.42,23.2,11.77,18.67,34.52,30.58,25.26,25.33,28.14,42.79,-59.15,35.28,60.1,34.4,-26.15,33.87,5.99,35

# Making EAZY Filter file

In [5]:
filtdir = WD/'data'/'fiducial_filters_102'
filters_spherex = make_eazy_filters_spherex(filtdir, 'FILTERS+spherex.res')

Wrote <FILTERS+spherex.res[.info]>


In [9]:
params = {}
params['FILTERS_RES']       = "FILTERS+spherex.res"
params['TEMPLATES_FILE']    = "BROWN_COSMOS.template"
params['WAVELENGTH_FILE ']  = "../lambda.def" # TODO: check this
params['TEMP_ERR_FILE']     = "../template_error_cosmos2020.txt" # TODO: check this

params['TEMPLATE_COMBOS']   = 1
params['TEMP_ERR_A2']       = 0.2
params['SYS_ERR']           = 0.01

params['MW_EBV']            = 0.012
params['CAT_HAS_EXTCORR']   = True

params['CATALOG_FILE']      = "data/spherex_synphot.dat" 

dir_output = WD/"output"

if not dir_output.exists():
    dir_output.mkdir()
params['OUTPUT_DIRECTORY']  = dir_output.as_posix()
params['MAIN_OUTPUT_FILE']  = "output"

params['APPLY_PRIOR']       = False 
params['PRIOR_FILE']        = "../prior_K_TAO.dat"
params['PRIOR_FILTER']      = 107  # 2MASS Ks band
params['PRIOR_ABZP']        = 23.9
params['PRIOR_FLOOR']       = 0.01

params['FIX_ZSPEC']         = False
params['Z_MIN']             = 0.0
params['Z_MAX']             = 6.0
params['Z_STEP']            = 0.003
params['Z_STEP_TYPE']       = 1

translate_filename = 'spherex.translate'

# Making EAZY `.template` file

In [51]:
lines = open("../BROWN_COSMOS.list").readlines()
with open(params['TEMPLATES_FILE'], 'w') as f:
    for i, line in enumerate(lines):
        if not line.startswith('#'):
            f.write(f'{i+1:3d} ../{line}')

# Making EAZY `.translate` file

In [8]:
ids_sphx = filters_spherex.search('SPHEREx')
nums_sphx = ids_sphx + 1

colnames = ['id']
for num in nums_sphx:
    colnames.append(f'F{num}')
    colnames.append(f'E{num}')

cat_synphot = Table.read(WD/'data/L3_COSMOS2020_Richard_2023DEC4_SPHEREx_fiducial102_for_photoz.txt',
                         format='ascii.no_header', names=colnames)
cat_synphot.write(WD/'data/spherex_synphot.dat', format='ascii.commented_header', overwrite=True)
del cat_synphot


with open(translate_filename, 'w') as f:
    for num in nums_sphx:
        f.write(f'F{num} F{num}\n')
        f.write(f'E{num} E{num}\n')


  419 SPHEREx_Band1_1 lambda_c= 6.9090e+03 AB-Vega= 0.360 w95=9992.6
  420 SPHEREx_Band1_2 lambda_c= 7.0989e+03 AB-Vega= 0.387 w95=9711.2
  421 SPHEREx_Band1_3 lambda_c= 7.4153e+03 AB-Vega= 0.424 w95=9346.2
  422 SPHEREx_Band1_4 lambda_c= 7.4454e+03 AB-Vega= 0.452 w95=9084.0
  423 SPHEREx_Band1_5 lambda_c= 7.5601e+03 AB-Vega= 0.484 w95=8761.8
  424 SPHEREx_Band1_6 lambda_c= 7.9043e+03 AB-Vega= 0.508 w95=8682.3
  425 SPHEREx_Band1_7 lambda_c= 7.9427e+03 AB-Vega= 0.553 w95=8377.5
  426 SPHEREx_Band1_8 lambda_c= 8.0093e+03 AB-Vega= 0.535 w95=8026.9
  427 SPHEREx_Band1_9 lambda_c= 8.6532e+03 AB-Vega= 0.550 w95=7737.0
  428 SPHEREx_Band1_10 lambda_c= 8.5533e+03 AB-Vega= 0.518 w95=7399.9
  429 SPHEREx_Band1_11 lambda_c= 9.2650e+03 AB-Vega= 0.561 w95=7028.0
  430 SPHEREx_Band1_12 lambda_c= 8.9319e+03 AB-Vega= 0.545 w95=6689.0
  431 SPHEREx_Band1_13 lambda_c= 9.6759e+03 AB-Vega= 0.572 w95=1186.8
  432 SPHEREx_Band1_14 lambda_c= 9.0150e+03 AB-Vega= 0.620 w95=374.5
  433 SPHEREx_Band1_15 lambda_

In [7]:
import gc
gc.collect()

0

In [ ]:
ez = eazy.photoz.PhotoZ(param_file=None, translate_file=translate_filename,
                        zeropoint_file=None, params=params, n_proc=20,
                        timeout=3600)

with open('tempfilt.pickle', 'wb') as wfile:
   pickle.dump(ez.tempfilt, wfile, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
ez.fit_catalog(ez.idx, n_proc=8)

ez.show_fit(id=0, savefig='fit.png')

In [ ]:
with open('tempfilt.pickle', 'rb') as rfile:
    tempfilt = pickle.load(rfile)
ez = eazy.photoz.PhotoZ(param_file=None, translate_file='test.translate', zeropoint_file=None, params=params, tempfilt=tempfilt, )
ez.fit_catalog(ez.idx, n_proc=8)

In [ ]:
zout, hdu = ez.standard_output(prior=True, beta_prior=False)
zout.write('output.fits', overwrite=True)